<a href="https://colab.research.google.com/github/NDVDS/YOLOv5-Cars-detecting/blob/main/Copy_of_CarsDetecting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/NDVDS/YOLOv5-Cars-detecting.git
%cd /content/YOLOv5-Cars-detecting

Cloning into 'YOLOv5-Cars-detecting'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 59 (delta 21), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (59/59), done.
/content/YOLOv5-Cars-detecting


In [3]:
#Install requirements and load pretrained yolov5s model
!pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

import torch
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained = True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 182 kB 14.1 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 1.6 MB 56.2 MB/s 


/usr/local/lib/python3.8/dist-packages/torch/hub.py:266: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2022-12-4 Python-3.8.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)



  0%|          | 0.00/14.1M [00:00<?, ?B/s]


Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [4]:
#Modifying calsses and confidence parameters
model.classes = 2 #Cars only
model.conf = 0.73

In [7]:
im = '/content/YOLOv5-Cars-detecting/imageSample.jpg'

In [8]:
#Model inference and printing results in json format
results = model(im)
bboxes = results.pandas().xyxy[0][['xmin','ymin','xmax','ymax']]
print(bboxes)

          xmin         ymin         xmax         ymax
0   104.034012   596.988647   958.334778   937.376587
1  2272.018555   663.893738  3152.115234  1003.014771
2  2309.560791    26.014078  3009.581299   506.093903
3  2415.671631  1136.792847  3084.042236  1511.921631
4  1368.518555  1137.739136  1915.031250  1554.121582
5  1326.271118    69.331818  1928.485352   447.571075
6    98.914406    33.905552  1035.887207   493.957031
7   133.226410  1121.580078   974.256531  1516.974487


In [ ]:
results.show()

In [ ]:
#Saving JSON
bboxes.to_json('BBOXESPred.json',orient='index')

In [ ]:
#VIDEO:
#Extracting farme by frame foward to model input,
#writing results to new video.
import cv2
from tqdm import tqdm

vid_cap = cv2.VideoCapture('/content/YOLOv5-Cars-detecting/videoInput.mp4')
frame_read, img = vid_cap.read()
      
fps = vid_cap.get(cv2.CAP_PROP_FPS) 
w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_count = int(vid_cap.get(cv2.CAP_PROP_FRAME_COUNT))
out = cv2.VideoWriter('/content/YOLOv5-Cars-detecting/videoOutput.mp4', cv2.VideoWriter_fourcc(*'MP4V'), fps, (w,h))

while(frame_read):
  for i in tqdm(range(frame_count)):
    result = model(img)
    im = result.render()[0]
    out.write(im)
    frame_read, img = vid_cap.read()
    
out.release()
vid_cap.release()